#### Data analysis notebook

Reads in the data files and plot results



In [ ]:
from RunManager import RunManager
from Geant4Analyzer import Geant4Analyzer
from XAMSPlotter import XAMSPlotter
import matplotlib.pyplot as plt

##### Initialize RunManager

Reads the runs database and acts as a bookkeeper for the simulations. You can then select runs based on their id's and analyze those simulations

In [ ]:
manager = RunManager("../run/rundb.json")
display(manager.display_all_runs(include_deleted=False))

##### Process

Processing of the selected data. Here also cuts are defined:
- **cut** are the cuts on the event/detector level
- **cut_hit** are he cuts on the individual hits (actually a hit here is a cluster made in G4Sim)

In [ ]:
# cut on the global event variables
gxe = 0 # gaseous xenon
lxe = 1 # liquid xenon
nai = 2 # NaI detector

cut = lambda data: (data['ndet'][:,lxe] ==1)
# additional cuts on the clusters
cut_hit = lambda data: (data['eh'] > 0.) & (data['id'] >-1)

std = Geant4Analyzer("run_05", first_only=True)
std.preprocess_data(cut=cut, cut_hit=cut_hit)


Now you can play with the selected data. In the 2D distributions you have the possibility to plot the detector geometry superimposed to the data. 

In [ ]:
# 2D plot with log color scale
from matplotlib.colors import LogNorm

fig, ax = plt.subplots()
fig.set_size_inches(5,5)

h= ax.hist2d(std.data['xh'], std.data['yh'], bins=500, range=[[-200,200],[-100,300]], cmap='viridis', norm=LogNorm())

gpl = XAMSPlotter(std.geometry)
gpl.plot_geometry(ax=ax, view = 'xy')
fig.savefig("xams_xy.pdf")

plt.show()


In [ ]:
# 2D plot with log color scale
import json
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
from XAMSPlotter import XAMSPlotter


fig, ax = plt.subplots()
fig.set_size_inches(5,5)


h= ax.hist2d(std.data['r'], std.data['zh'], bins=500, range=[[0,500],[-250,250]], cmap='viridis', norm=LogNorm())
ax.plot([0,400],[-25., -25.],'--',color='grey',linewidth=0.5)

gpl = XAMSPlotter(std.geometry)
gpl.plot_geometry(ax=ax, view = 'rz')

fig.savefig("xams_rz.pdf")
plt.show()


In [ ]:
std1=std

cut_hit = lambda data: (data['eh'] > 0.) & (data['id'] == 1) & data['r'] < 55 & data['zh'] > -60 & data['zh'] < -10
std1.preprocess_data(cut=cut, cut_hit=cut_hit)
std1.plot_histogram('eh', bins=1000)

In [ ]:
h = plt.hist(std1.data['eh'], bins=1000, histtype='step', color='black')

In [ ]:
import numpy as np
# Assuming std1.data['eh'] is your original energy data
data = std1.data['eh']

# Calculate the Gaussian noise to add to each data point
# 0.05 * data represents 5% energy resolution
noise = np.random.normal(0, 0.10 * data, size=data.shape)

# Add the noise to the original data to simulate smearing
smeared_data = data + noise

bins = 100
# Plot the original histogram
plt.hist(data, bins=bins, histtype='step', color='black', label='Geant4 clusters')

# Plot the smeared histogram
plt.hist(smeared_data, bins=bins, histtype='step', color='red', linestyle='solid', label='Smeared')
plt.yscale('log')
plt.xlabel('Energy (keV)')
plt.legend(frameon=False)

plt.savefig("energy_resolution.pdf")